# Import Functions

In [35]:
import pandas as pd
import numpy as np
from small_world_functions import sub_df, load_main_monsters, df_to_small_world_adjacency_matrix, ydk_to_monster_names

# Load Cards and Create Adjacency Matrix

In [36]:
MAIN_MONSTERS = load_main_monsters()
SW_ADJACENCY_MATRIX = df_to_small_world_adjacency_matrix(MAIN_MONSTERS) #small world adjacency matrix of all cards

# Find Best Bridges

In [37]:
def find_best_bridges(deck_monster_names, required_target_names=[]):
    #inputs: list of monster names and list of monsters that are required to connect with the small world bridges
    #output: The bridges that connect the most cards in your deck and connect with all the required targets
    deck_monster_names = list(set(deck_monster_names) | set(required_target_names)) #union names
    deck_indices = sub_df(MAIN_MONSTERS, deck_monster_names, 'name').index #indices of deck monsters
    required_indices = sub_df(MAIN_MONSTERS, required_target_names, 'name').index #indices of required targets

    num_required_targets = len(required_target_names) #number of cards required to connect with one bridge

    required_monster_matrix = SW_ADJACENCY_MATRIX[required_indices,:] #array corresponding to (required connection monsters) by (all monsters)
    num_bridges_to_required_cards = required_monster_matrix.sum(axis=0) #number of required connections satisfied by all monsters
    required_bridge_mask = num_bridges_to_required_cards==num_required_targets
    df_bridges = MAIN_MONSTERS[required_bridge_mask].copy() #data frame of monsters connecting all required targets
    required_bridge_indices = df_bridges.index #indices of monsters that satisfy all required connections
    if len(df_bridges)==0:
        print('There are no monsters that bridge all required targets.')
        return

    #subset of adjacency matrix corresponding to (deck monsters) by (monsters with connections to the required cards)
    bridge_matrix = SW_ADJACENCY_MATRIX[deck_indices,:][:,required_bridge_indices]

    num_deck_bridges = bridge_matrix.sum(axis=0)
    df_bridges['number_of_connections'] = num_deck_bridges
    df_bridges = df_bridges[df_bridges['number_of_connections'] > 0]
    df_bridges = df_bridges[['number_of_connections', 'name', 'type', 'attribute', 'level', 'atk', 'def']] #reorder columns
    df_bridges = df_bridges.sort_values(by=['number_of_connections','name'], ascending=[False, True]).reset_index(drop=True) #reorder rows
    return df_bridges

def find_best_bridges_from_ydk(ydk_file):
    deck_monster_names = ydk_to_monster_names(ydk_file, MAIN_MONSTERS)
    bridges = find_best_bridges(deck_monster_names)
    return bridges

## Bridge Finder Examples

### List of Names

In [38]:
#cards that we want to find bridges between
deck_monster_names = ['Ash Blossom & Joyous Spring',
                        'Effect Veiler',
                        'Ghost Belle & Haunted Mansion',
                        'Mathmech Addition',
                        'Mathmech Circular',
                        'Mathmech Diameter',
                        'Mathmech Multiplication',
                        'Mathmech Nabla',
                        'Mathmech Sigma',
                        'Mathmech Subtraction',
                        'Parallel eXceed',
                        'Maxx "C"']

#cards that are required to connect with a bridge
required_target_names = ['Mathmech Circular',
                        'Maxx "C"',
                        'Parallel eXceed']

bridges = find_best_bridges(deck_monster_names, required_target_names)
bridges.head(20)

,number_of_connections,name,type,attribute,level,atk,def
0,10.0,Boot-Up Corporal - Command Dynamo,Machine,EARTH,4.0,0.0,2000.0
1,10.0,Boot-Up Soldier - Dread Dynamo,Machine,EARTH,4.0,0.0,2000.0
2,10.0,Doshin @Ignister,Cyberse,EARTH,1.0,100.0,800.0
3,10.0,Flowerdino,Dinosaur,EARTH,4.0,2000.0,0.0
4,10.0,Gem-Turtle,Rock,EARTH,4.0,0.0,2000.0
5,10.0,Gogogo Giant,Rock,EARTH,4.0,2000.0,0.0
6,10.0,Heroic Challenger - Swordshield,Warrior,EARTH,4.0,0.0,2000.0
7,10.0,Knightmare Corruptor Iblee,Cyberse,DARK,2.0,0.0,0.0
8,10.0,Madolche Marmalmaide,Spellcaster,EARTH,4.0,800.0,2000.0
9,10.0,Miracle Jurassic Egg,Dinosaur,EARTH,4.0,0.0,2000.0


### From YDK File

In [39]:
ydk_file = 'mathmech_deck.ydk'
bridges_from_ydk = find_best_bridges_from_ydk(ydk_file) #find the best Small World bridges for your deck
bridges_from_ydk.head(20)

,number_of_connections,name,type,attribute,level,atk,def
0,11.0,Adamancipator Crystal - Raptite,Rock,WIND,4.0,0.0,2200.0
1,11.0,Alchemist of Black Spells,Spellcaster,WIND,4.0,1200.0,1800.0
2,11.0,Backlinker,Cyberse,DARK,3.0,1600.0,0.0
3,11.0,Guard Penguin,Winged Beast,WATER,4.0,0.0,1200.0
4,11.0,Mixeroid,Machine,WIND,4.0,0.0,2200.0
5,11.0,Shinobird Crow,Winged Beast,WIND,4.0,0.0,0.0
6,11.0,Skull Conductor,Zombie,DARK,4.0,2000.0,0.0
7,11.0,Yosenju Shinchu L,Rock,WIND,4.0,0.0,2100.0
8,11.0,Yosenju Shinchu R,Rock,WIND,4.0,0.0,2100.0
9,10.0,Adamancipator Crystal - Dragite,Rock,WATER,4.0,0.0,2200.0


# Small World Adjacency Matrix

In [40]:
def monster_names_to_df(card_names):
    #input: list of card names. output: dataframe of card names
    df_cards = sub_df(MAIN_MONSTERS, card_names, 'name')
    return df_cards

def ydk_to_df_adjacency_matrix(ydk_file, squared=False):
    #input: ydk file of deck. Optional parameter to square resulting matrix
    #output: adjacency matrix dataframe
    card_names = ydk_to_monster_names(ydk_file, MAIN_MONSTERS)
    df_cards = monster_names_to_df(card_names)
    adjacency_matrix = df_to_small_world_adjacency_matrix(df_cards)
    if squared==True:
        adjacency_matrix = np.linalg.matrix_power(adjacency_matrix, 2)
    df_adjacency_matrix = pd.DataFrame(adjacency_matrix, index=card_names, columns=card_names)
    return df_adjacency_matrix

## Adjacency Matrix Example

In [41]:
mathmech_adjacency_matrix = ydk_to_df_adjacency_matrix(ydk_file)
#if an entry corresponding to two cards is non-zero, then one card connects to the other with Small World
mathmech_adjacency_matrix

,Ash Blossom & Joyous Spring,D.D. Crow,Effect Veiler,Ghost Belle & Haunted Mansion,Mathmech Addition,Mathmech Circular,Mathmech Diameter,Mathmech Multiplication,Mathmech Nabla,Mathmech Sigma,Mathmech Subtraction,Parallel eXceed
Ash Blossom & Joyous Spring,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
D.D. Crow,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Effect Veiler,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
Ghost Belle & Haunted Mansion,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Mathmech Addition,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Mathmech Circular,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Mathmech Diameter,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Mathmech Multiplication,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mathmech Nabla,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Mathmech Sigma,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [42]:
mathmech_adjacency_matrix_squared = ydk_to_df_adjacency_matrix(ydk_file, squared=True)
#if an entry corresponding to two cards is non-zero, then one card is searchable from the other with Small World
mathmech_adjacency_matrix_squared

,Ash Blossom & Joyous Spring,D.D. Crow,Effect Veiler,Ghost Belle & Haunted Mansion,Mathmech Addition,Mathmech Circular,Mathmech Diameter,Mathmech Multiplication,Mathmech Nabla,Mathmech Sigma,Mathmech Subtraction,Parallel eXceed
Ash Blossom & Joyous Spring,3.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0
D.D. Crow,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
Effect Veiler,0.0,0.0,6.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,3.0
Ghost Belle & Haunted Mansion,1.0,1.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
Mathmech Addition,0.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0
Mathmech Circular,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0
Mathmech Diameter,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0
Mathmech Multiplication,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Mathmech Nabla,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0
Mathmech Sigma,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,2.0,1.0,0.0
